目标:
1. 使用CSV文件作为数据的来源，采用train_and_eval(keras自带在train当中)进行训练，
2. 进而使用数据验证模型，测试数据测试模型，然后运行一下预测；最后导出模型供使用。

In [193]:
import tensorflow as tf
import shutil
import math
from datetime import datetime
import multiprocessing
from tensorflow.python.feature_column import feature_column
sess = tf.Session()

In [194]:
MODEL_NAME = 'reg-model-02'

TRAIN_DATA_FILES_PATTERN = 'data/train-*.csv'
VALID_DATA_FILES_PATTERN = 'data/valid-*.csv'
TEST_DATA_FILES_PATTERN = 'data/test-*.csv'

RESUME_TRAINING = False
PROCESS_FEATURES = True
EXTEND_FEATURE_COLUMNS = True
MULTI_THREADING = True

# 基本步骤
1. 定义数据集的元数据(常量到处使用)
2. 定义读取CSV的输入函数及其解析(ETL)
3. 定义特征列
4. 定义估计器的创建函数
5. 运行试验，包括运行方案的定义
6. 运行模型的评估
7. 执行预测和部署已存的模型



## 1. 定义数据集元数据
- CSV的头和默认值
- 数据和类别特征名称
- 目标列名称
- 无用列名称

In [195]:
HEADER = ['key','x','y','alpha','beta','target']
HEADER_DEFAULTS = [[0], [0.0], [0.0], ['NA'], ['NA'], [0.0]]
NUMERIC_FEATURE_NAMES = ['x', 'y']  
CATEGORICAL_FEATURE_NAMES_WITH_VOCABULARY = {'alpha':['ax01', 'ax02'], 'beta':['bx01', 'bx02']}
CATEGORICAL_FEATURE_NAMES = list(CATEGORICAL_FEATURE_NAMES_WITH_VOCABULARY.keys())
FEATURE_NAMES = NUMERIC_FEATURE_NAMES + CATEGORICAL_FEATURE_NAMES
TARGET_NAME = 'target'
UNUSED_FEATURE_NAMES = list(set(HEADER) - set(FEATURE_NAMES) - {TARGET_NAME})

## 2. 定义数据数据输入函数
- 输入文件名称模式
- 使用dataset读入数据
- 解析特征
- 使用处理
- 返回(特征，目标)张量


In [196]:
def process_features(features):
    """输入数据时张量，采用tf-api，特征名-张量的字典"""
    features['x_2']=tf.square(features['x'])
    features["x_2"] = tf.square(features['x'])
    features["y_2"] = tf.square(features['y'])
    features["xy"] = tf.multiply(features['x'], features['y']) # features['x'] * features['y']
    features['dist_xy'] =  tf.sqrt(tf.squared_difference(features['x'],features['y']))
    return features

def parse_csv_row(csv_row):
    """返回特征字典-目标元组，供dataset"""
    columns=tf.decode_csv(csv_row,record_defaults=HEADER_DEFAULTS) #按顺序解析
    features=dict(zip(HEADER,columns))
    for col in UNUSED_FEATURE_NAMES:
        features.pop(col)
    target= features.pop(TARGET_NAME)
    return features,target
    
    
def csv_input_fn(files_name_pattern, mode=tf.estimator.ModeKeys.EVAL, 
                 skip_header_lines=0,num_epochs=None,batch_size=200):
    
    shuffle=True if mode==tf.estimator.ModeKeys.TRAIN else False
    input_file_names=tf.matching_files(files_name_pattern)
    dataset = tf.data.TextLineDataset(input_file_names)
    dataset = dataset.skip(skip_header_lines)
    dataset = dataset.map(parse_csv_row)
    if PROCESS_FEATURES:
        dataset = dataset.map(lambda features, target: (process_features(features), target))
    if shuffle:
        dataset = dataset.shuffle(buffer_size=2 * batch_size + 1)
    dataset = dataset.batch(batch_size)
    dataset = dataset.repeat(num_epochs)
    iterator = dataset.make_one_shot_iterator()
    features,target =iterator.get_next()
    return features,target
# 读取文件的输出列和特征列feature_columns输入列不容易对上，这个地方最好共享一份数据，而不是人工重写核对
features, target = csv_input_fn(files_name_pattern="")
print("Feature read from CSV: {}".format(list(features.keys())))
print("Target read from CSV: {}".format(target))

Feature read from CSV: ['x', 'y', 'alpha', 'beta', 'x_2', 'y_2', 'xy', 'dist_xy']
Target read from CSV: Tensor("IteratorGetNext_15:8", shape=(?,), dtype=float32)


### 3.定义特征列
假设数值列规范化了或者同尺度，否则使用特征列构造器，
传递normlizer_fn及其normlisation params

In [197]:
def extend_feature_columns(feature_columns):
    """添加交叉列"""
    feature_columns['alpha_X_beta'] = tf.feature_column.crossed_column([feature_columns['alpha'], feature_columns['beta']], 4)
    return feature_columns

def get_feature_columns():
    CONSTRUCTED_NUMERIC_FEATURES_NAMES = ['x_2', 'y_2', 'xy', 'dist_xy']
    all_numeric_feature_names = NUMERIC_FEATURE_NAMES.copy() 
    if PROCESS_FEATURES: # 进一步选择输入的列
        all_numeric_feature_names += CONSTRUCTED_NUMERIC_FEATURES_NAMES
    numeric_columns = {feature_name: tf.feature_column.numeric_column(feature_name)
                       for feature_name in all_numeric_feature_names}
    categorical_column_with_vocabulary = \
        {item[0]: tf.feature_column.categorical_column_with_vocabulary_list(item[0], item[1])
         for item in CATEGORICAL_FEATURE_NAMES_WITH_VOCABULARY.items()}
    # 将准备好的列全出整起来
    feature_columns = {}
    if numeric_columns is not None:
        feature_columns.update(numeric_columns)
    if categorical_column_with_vocabulary is not None:
        feature_columns.update(categorical_column_with_vocabulary)
    # 函数和常量一样是个模块全局对象没有必要传递，直接引用就好
    if EXTEND_FEATURE_COLUMNS:
        feature_columns = extend_feature_columns(feature_columns)
    return feature_columns

feature_columns = get_feature_columns()
print("Feature Columns: {}".format(feature_columns))

Feature Columns: {'x': _NumericColumn(key='x', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), 'y': _NumericColumn(key='y', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), 'x_2': _NumericColumn(key='x_2', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), 'y_2': _NumericColumn(key='y_2', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), 'xy': _NumericColumn(key='xy', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), 'dist_xy': _NumericColumn(key='dist_xy', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), 'alpha': _VocabularyListCategoricalColumn(key='alpha', vocabulary_list=('ax01', 'ax02'), dtype=tf.string, default_value=-1, num_oov_buckets=0), 'beta': _VocabularyListCategoricalColumn(key='beta', vocabulary_list=('bx01', 'bx02'), dtype=tf.string, default_value=-1, num_oov_buckets=0), 'alpha_X_beta': _CrossedColumn(keys=(_VocabularyListCategoricalColumn(key=

In [198]:
feature_columns.keys()

dict_keys(['x', 'y', 'x_2', 'y_2', 'xy', 'dist_xy', 'alpha', 'beta', 'alpha_X_beta'])

In [199]:
for i in dense_columns+categorical_columns:
    print(i.key)

x
y
x_2
y_2
xy
dist_xy
alpha
beta


### 4. 定义估计器创建函数
- 从特征列中获取数值特征列
- 将类型特征列转化成
- 使用dense + indicator feature columns + params创建估计器

In [200]:
def create_estimator(run_config,hparams):
    feature_columns = list(get_feature_columns().values())
    dense_columns=list(filter(lambda col:isinstance(col,feature_column._NumericColumn),feature_columns))
    categorical_columns=list(filter(lambda col:isinstance(col,(feature_column._VocabularyListCategoricalColumn,feature_column._BucketizedColumn)),
                                  feature_columns))
    indicator_columns = list(map(lambda column: tf.feature_column.indicator_column(column),categorical_columns))
    
    estimator=tf.estimator.DNNRegressor(feature_columns=dense_columns+indicator_columns,
                                       hidden_units=hparams.hidden_units,optimizer=tf.train.AdadeltaOptimizer(),
                                       activation_fn=tf.nn.elu,dropout= hparams.dropout_prob,config= run_config)
    return estimator

In [202]:
list(get_feature_columns().keys())
dense_columns+indicator_columns

[_NumericColumn(key='x', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='y', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='x_2', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='y_2', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='xy', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='dist_xy', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _IndicatorColumn(categorical_column=_VocabularyListCategoricalColumn(key='alpha', vocabulary_list=('ax01', 'ax02'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 _IndicatorColumn(categorical_column=_VocabularyListCategoricalColumn(key='beta', vocabulary_list=('bx01', 'bx02'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [203]:
feature_columns = list(get_feature_columns().values())
dense_columns=list(filter(lambda col:isinstance(col,feature_column._NumericColumn),feature_columns))
categorical_columns=list(filter(lambda col:isinstance(col,(feature_column._VocabularyListCategoricalColumn,feature_column._BucketizedColumn)),
                                  feature_columns))
indicator_columns = list(map(lambda column: tf.feature_column.indicator_column(column),categorical_columns))

dense_columns+indicator_columns

[_NumericColumn(key='x', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='y', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='x_2', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='y_2', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='xy', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='dist_xy', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _IndicatorColumn(categorical_column=_VocabularyListCategoricalColumn(key='alpha', vocabulary_list=('ax01', 'ax02'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 _IndicatorColumn(categorical_column=_VocabularyListCategoricalColumn(key='beta', vocabulary_list=('bx01', 'bx02'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [204]:
cl=dense_columns[0]
cl.key

'x'

### 5. 运行试验
- a. 设置运行参数和模型超参数HParam and RunConfig
- b. 定义Serving Function
- c. 定义一个Early Stopping Monitor (Hook)
- d. 定义TrainSpec and EvaluSpec方案

进行了这个多定义，现在模块空间有多少对象？函数对象和实例对象

In [ ]:
TRAIN_SIZE = 12000
NUM_EPOCHS = 1000
BATCH_SIZE = 500
EVAL_AFTER_SEC = 15
TOTAL_STEPS = (TRAIN_SIZE/BATCH_SIZE)*NUM_EPOCHS

hparams  = tf.contrib.training.HParams(
    num_epochs = 100, 
    batch_size = 500,
    hidden_units=[8, 4],
    early_stopping_rounds = 1,
    dropout_prob = 0.0)

model_dir = 'trained_models/{}'.format(MODEL_NAME)

run_config = tf.estimator.RunConfig((
    save_checkpoints_steps=480, # to evaluate after each 20 epochs => (12000/500) * 20
    tf_random_seed=19830610,
    model_dir=model_dir
)

print("Model directory: {}".format(run_config.model_dir))
print("Hyper-parameters: {}".format(hparams))
print("")


In [ ]:
def csv_serving_input():
    SERVING_HEADER = ['x','y','alpha','beta']
    SERVING_HEADER_DEFAULTS = [[0.0], [0.0], ['NA'], ['NA']]
    rows_string_tensor = tf.placeholder(dtype=tf.string,shape=[None], name='csv_rows')
    receiver_tensor = {'csv_rows': rows_string_tensor}
    row_columns = tf.expand_dims(rows_string_tensor, -1) #变成2阶，另一axis来放置特征
    columns = tf.decode_csv(row_columns, record_defaults=SERVING_HEADER_DEFAULTS)
    features = dict(zip(SERVING_HEADER, columns))
    if PROCESS_FEATURES: features = process_features(features)
    return tf.estimator.export.ServingInputReceiver(features, receiver_tensor)
    

In [ ]:
class EarlyStoppingHook(tf.train.SessionRunHook):
    def __init__(self, early_stopping_rounds=1):
        self._best_loss = None
        self._early_stopping_rounds = early_stopping_rounds
        self._counter = 0
        print("*** Early Stopping Hook: - Created")
        print("*** Early Stopping Hook:: Early Stopping Rounds: {}".format(self._early_stopping_rounds))
    def before_run(self, run_context):  # 调度程序返回给方法的对象
        graph = run_context.session.graph
        loss_tensor = graph.get_collection(tf.GraphKeys.LOSSES)[1]
        return tf.train.SessionRunArgs(loss_tensor)

    def after_run(self, run_context, run_values):
        last_loss = run_values.results
        print("************************")
        print("** Evaluation Monitor - Early Stopping **")
        print("Early Stopping Hook: Current loss: {}".format(str(last_loss)))
        print("Early Stopping Hook: Best loss: {}".format(str(self._best_loss)))

        if self._best_loss is None:
            self._best_loss = last_loss
        elif last_loss > self._best_loss:
            self._counter += 1
            print("Early Stopping Hook: No improvment! Counter: {}".format(self._counter))
            if self._counter == self._early_stopping_rounds:
                run_context.request_stop()
                print("Early Stopping Hook: Stop Requested: {}".format(run_context.stop_requested))
        else:
            self._best_loss = last_loss
            self._counter = 0
        print("************************")

In [223]:
import functools
train_input_fn=functools.partial(csv_input_fn,
                                 files_name_pattern=TRAIN_DATA_FILES_PATTERN,
                                 mode=tf.estimator.ModeKeys.TRAIN,
                                 num_epochs=hparams.num_epochs,
                                 batch_size=hparams.batch_size)

valid_input_fn =functools.partial(csv_input_fn,
                                  files_name_pattern=VALID_DATA_FILES_PATTERN,
                                  mode=tf.estimator.ModeKeys.EVAL,
                                  num_epochs=1,
                                  batch_size=hparams.batch_size)

test_input_fn =functools.partial(csv_input_fn,
                                  files_name_pattern=TEST_DATA_FILES_PATTERN,
                                  mode=tf.estimator.ModeKeys.EVAL,
                                  num_epochs=hparams.num_epochs,
                                  batch_size=hparams.batch_size)

predict_input_fn =functools.partial(csv_input_fn,
                                    files_name_pattern= TEST_DATA_FILES_PATTERN,
                                    mode= tf.estimator.ModeKeys.PREDICT,batch_size= 5)


train_spec=tf.estimator.TrainSpec(train_input_fn,max_steps=hparams.max_steps,hooks=None)



eval_spec = tf.estimator.EvalSpec(valid_input_fn,
                                  steps=None,
                                  throttle_secs = EVAL_AFTER_SEC)# evalute after each 15 training seconds!

In [221]:
##############################所使用的对象定义(定义函数即实例化函数对象)完毕，下面开始训练操作####################

In [215]:
if not RESUME_TRAINING:
    print("Removing previous artifacts...")
    shutil.rmtree(model_dir, ignore_errors=True)
else:
    print("Resuming training...") 
time_start = datetime.utcnow() 
tf.logging.set_verbosity(tf.logging.INFO)
print("Experiment started at {}".format(time_start.strftime("%H:%M:%S")))
print(".......................................") 
estimator=create_estimator(run_config,hparams)
tf.estimator.train_and_evaluate(estimator=estimator,train_spec=train_spec,eval_spec=eval_spec)
time_end=datetime.utcnow()
print(".......................................")
print("Experiment finished at {}".format(time_end.strftime("%H:%M:%S")))
print("")
time_elapsed = time_end - time_start
print("Experiment elapsed time: {} seconds".format(time_elapsed.total_seconds()))


Removing previous artifacts...
Experiment started at 11:26:37
.......................................
INFO:tensorflow:Using config: {'_model_dir': 'trained_models/reg-model-02', '_tf_random_seed': 19830610, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000000040547358>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Not using Distribute Coordinator.
INFO:ten

INFO:tensorflow:global_step/sec: 49.6032
INFO:tensorflow:loss = 147670.06, step = 6801 (2.015 sec)
INFO:tensorflow:global_step/sec: 49.0918
INFO:tensorflow:loss = 154400.98, step = 6901 (2.037 sec)
INFO:tensorflow:global_step/sec: 49.6032
INFO:tensorflow:loss = 152946.77, step = 7001 (2.016 sec)
INFO:tensorflow:global_step/sec: 49.975
INFO:tensorflow:loss = 137715.98, step = 7101 (2.001 sec)
INFO:tensorflow:global_step/sec: 47.0588
INFO:tensorflow:loss = 183917.7, step = 7201 (2.125 sec)
INFO:tensorflow:global_step/sec: 48.1464
INFO:tensorflow:loss = 156468.45, step = 7301 (2.078 sec)
INFO:tensorflow:global_step/sec: 50.813
INFO:tensorflow:loss = 160316.95, step = 7401 (1.967 sec)
INFO:tensorflow:global_step/sec: 50.3778
INFO:tensorflow:loss = 165363.77, step = 7501 (1.985 sec)
INFO:tensorflow:global_step/sec: 47.9846
INFO:tensorflow:loss = 155529.89, step = 7601 (2.084 sec)
INFO:tensorflow:global_step/sec: 48.4731
INFO:tensorflow:loss = 156363.02, step = 7701 (2.063 sec)
INFO:tensorfl

INFO:tensorflow:global_step/sec: 48.3793
INFO:tensorflow:loss = 139738.2, step = 15101 (2.067 sec)
INFO:tensorflow:global_step/sec: 49.456
INFO:tensorflow:loss = 156890.0, step = 15201 (2.023 sec)
INFO:tensorflow:global_step/sec: 45.6621
INFO:tensorflow:loss = 153199.52, step = 15301 (2.189 sec)
INFO:tensorflow:global_step/sec: 48.0307
INFO:tensorflow:loss = 153206.05, step = 15401 (2.083 sec)
INFO:tensorflow:global_step/sec: 46.8823
INFO:tensorflow:loss = 146254.62, step = 15501 (2.133 sec)
INFO:tensorflow:global_step/sec: 50.9684
INFO:tensorflow:loss = 181576.92, step = 15601 (1.961 sec)
INFO:tensorflow:global_step/sec: 47.8011
INFO:tensorflow:loss = 161736.88, step = 15701 (2.092 sec)
INFO:tensorflow:global_step/sec: 49.554
INFO:tensorflow:loss = 164578.77, step = 15801 (2.018 sec)
INFO:tensorflow:global_step/sec: 49.6524
INFO:tensorflow:loss = 161756.28, step = 15901 (2.014 sec)
INFO:tensorflow:global_step/sec: 49.505
INFO:tensorflow:loss = 144582.03, step = 16001 (2.020 sec)
INFO:

INFO:tensorflow:loss = 158495.14, step = 23301 (1.987 sec)
INFO:tensorflow:global_step/sec: 49.6524
INFO:tensorflow:loss = 178494.39, step = 23401 (2.015 sec)
INFO:tensorflow:global_step/sec: 50
INFO:tensorflow:loss = 144720.81, step = 23501 (2.000 sec)
INFO:tensorflow:global_step/sec: 48.0769
INFO:tensorflow:loss = 165223.75, step = 23601 (2.079 sec)
INFO:tensorflow:global_step/sec: 49.5786
INFO:tensorflow:loss = 159912.17, step = 23701 (2.017 sec)
INFO:tensorflow:global_step/sec: 48.7329
INFO:tensorflow:loss = 165833.47, step = 23801 (2.052 sec)
INFO:tensorflow:global_step/sec: 48.7329
INFO:tensorflow:loss = 154753.6, step = 23901 (2.053 sec)
INFO:tensorflow:Saving checkpoints for 24000 into trained_models/reg-model-02\model.ckpt.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-11-22-11:34:52
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from trained_models/reg-model-02\model.ckpt-24000
I

KeyboardInterrupt: 

In [218]:
train_results = estimator.evaluate(input_fn=train_input_fn, steps=1)
train_rmse = round(math.sqrt(train_results["average_loss"]),5)
print()
print("############################################################################################")
print("# Train RMSE: {} - {}".format(train_rmse, train_results))
print("############################################################################################")

valid_results = estimator.evaluate(input_fn=valid_input_fn, steps=1)
valid_rmse = round(math.sqrt(valid_results["average_loss"]),5)
print()
print("############################################################################################")
print("# Valid RMSE: {} - {}".format(valid_rmse,valid_results))
print("############################################################################################")

test_results = estimator.evaluate(input_fn=test_input_fn, steps=1)
test_rmse = round(math.sqrt(test_results["average_loss"]),5)
print()
print("############################################################################################")
print("# Test RMSE: {} - {}".format(test_rmse, test_results))
print("############################################################################################")

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-11-22-11:38:07
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from trained_models/reg-model-02\model.ckpt-24000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [1/1]
INFO:tensorflow:Finished evaluation at 2018-11-22-11:38:07
INFO:tensorflow:Saving dict for global step 24000: average_loss = 372.95963, global_step = 24000, label/mean = 1.7754534, loss = 186479.81, prediction/mean = 0.41338992
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 24000: trained_models/reg-model-02\model.ckpt-24000

############################################################################################
# Train RMSE: 19.31216 - {'average_loss': 372.95963, 'label/mean': 1.7754534, 'loss': 186479.81, 'prediction/mean': 0.41338992, 'global_step': 24000}
#######################################

In [224]:
import itertools


predictions = estimator.predict(input_fn=predict_input_fn)
values = list(map(lambda item: item["predictions"][0],list(itertools.islice(predictions, 5))))
print()
print("Predicted Values: {}".format(values))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from trained_models/reg-model-02\model.ckpt-24000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.

Predicted Values: [0.99982125, -0.011527352, -0.20706996, 0.13667259, 0.20198691]


In [225]:
import os

export_dir = model_dir +"/export/estimate"

saved_model_dir = export_dir + "/" + os.listdir(path=export_dir)[-1] 

print(saved_model_dir)

predictor_fn = tf.contrib.predictor.from_saved_model( export_dir = saved_model_dir, signature_def_key="predict")

output = predictor_fn({'csv_rows': ["0.5,1,ax01,bx02", "-0.5,-1,ax02,bx02"]})
print(output)

FileNotFoundError: [WinError 3] 系统找不到指定的路径。: 'trained_models/reg-model-02/export/estimate'

In [226]:
import os
os.getcwd()

'E:\\my_proj\\fog_recognition\\recognition_pre_fog_tf\\notebook'